In [1]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  750000 non-null  int64  
 1   id          750000 non-null  int64  
 2   Sex         750000 non-null  int64  
 3   Age         750000 non-null  int64  
 4   Height      750000 non-null  float64
 5   Weight      750000 non-null  float64
 6   Duration    750000 non-null  float64
 7   Heart_Rate  750000 non-null  float64
 8   Body_Temp   750000 non-null  float64
 9   Calories    750000 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 57.2 MB


## 特征说明
id 记录编号或用户编号
Sex 性别 0为男 1为女
Age 年龄
Height 身高（单位为 cm ）
Weight 体重（单位为 kg）
Duration  运动持续时间（分钟）
Heart_Rate 运动时的心率（BPM，心跳次数/分钟）
Body_Temp 运动时的体温（单位为 摄氏度 ）
Calories 运动过程中消耗的热量（通常以千卡 kcal 为单位）

## 绘制相关热图